# Chapter 5 - Recognizing music genres with TensorFlow and the Raspberry Pi Pico
## Part 1: Data collection and MFCCs computation

## Python libraries

In [ ]:
!pip install numpy==1.23.5
!pip install cmsisdsp==1.9.6

## Import libraries

In [ ]:
import soundfile as sf
import librosa
import numpy as np
import tensorflow as tf
import cmsisdsp as dsp
import math

from google.colab import drive

## Constants

In [ ]:
# Google drive top-level directory
G_DRIVE_ROOT = '/content/drive/'

# Audio sample rate
SAMPLE_RATE = 22050

# MFFC constants
FRAME_LENGTH = 2048
FRAME_STEP = 1024
FFT_LENGTH = 2048
FMIN_HZ = 20
FMAX_HZ = SAMPLE_RATE / 2
NUM_MEL_FREQS = 40
NUM_MFCCS = 18

## Extracting MFCCs from audio samples with TensorFlow


### Implement a function to compute MFCCs with the TensorFlow signal processing functions

In [ ]:
def extract_mfccs_tf(
  ad_src,
  ad_sample_rate,
  num_mfccs,
  frame_length,
  frame_step,
  fft_length,
  fmin_hz,
  fmax_hz,
  num_mel_freqs):

  n = ad_src.shape[0]
  num_frames = int(((n - frame_length) / frame_step) + 1)

  output = np.zeros(shape=(num_frames, num_mfccs))

  # Iterate over each frame to get the MFCC coefficients
  for i in range(num_frames):
    idx_s = i * frame_step
    idx_e = idx_s + frame_length
    src = ad_src[idx_s:idx_e]

    # Apply the Hann Window in-place
    hann_coef = tf.signal.hann_window(frame_length)
    hann = src * hann_coef

    # Apply the RFFT
    fft_spect = tf.signal.rfft(hann)

    # Calculate the magnitude of the FFT
    fft_mag_spect = tf.math.abs(fft_spect)

    # Calculate the Mel-spectrogram
    num_fft_freqs = fft_mag_spect.shape[0]
    mel_wei_mtx = tf.signal.linear_to_mel_weight_matrix(
      num_mel_freqs,
      num_fft_freqs,
      ad_sample_rate,
      fmin_hz,
      fmax_hz)

    # Calculate the Mel-spectrogram
    mel_spect = np.matmul(fft_mag_spect, mel_wei_mtx)

    # Calculate the Log-Mel-spectrogram
    log_mel_spect = np.log(mel_spect + 1e-6)

    dct = tf.signal.mfccs_from_log_mel_spectrograms(
    log_mel_spect)

    # Extract the MFFC coefficients
    output[i] = dct[0:num_mfccs]

  return output

### Mount top-level Google Drive

In [ ]:
drive.mount(G_DRIVE_ROOT)

In [ ]:
train_dir = "drive/MyDrive/mgr_dataset"

### Load an audio file

In [ ]:
filepath = train_dir + "/disco/disco.00002.wav"
ad, sr = sf.read(filepath)

print('Sample rate:', sr)
print('Sample shape:', ad.shape)
print('Song duration:', ad.shape[0] / sr)

### Play one-second of audio

In [ ]:
# Take one second of audio
test_ad = ad[0:SAMPLE_RATE]

import IPython.display as ipd
ipd.Audio(test_ad, rate=sr)

### Extract the MFCCs using TensorFlow

In [ ]:
mfccs_tf = extract_mfccs_tf(
    test_ad,
    SAMPLE_RATE,
    NUM_MFCCS,
    FRAME_LENGTH,
    FRAME_STEP,
    FFT_LENGTH,
    FMIN_HZ,
    FMAX_HZ,
    NUM_MEL_FREQS)

### Implement a function to visualize the MFCCs as image

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm

def display_mfccs(mfcc_src):
  fig, ax = plt.subplots()
  cax = ax.imshow(mfcc_src, interpolation='nearest', cmap=cm.gray, origin='lower')
  ax.set_title('MFCCs')
  plt.xlabel('Frame index - Time')
  plt.ylabel('Coefficient index - Frequency')
  plt.colorbar(cax)
  plt.savefig('filename.png', dpi=800)
  plt.show()

### Display the MFCCs

In [ ]:
display_mfccs(mfccs_tf.T)

### Implement a function to compute MFCCs algorithm with the Librosa library

In [ ]:
def extract_mfccs_librosa(
  ad_src,
  ad_sample_rate,
  num_mfccs,
  frame_length,
  frame_step,
  fft_length,
  fmin_hz,
  fmax_hz,
  num_mel_freqs):

  return librosa.feature.mfcc(
      y=ad_src,
      sr=ad_sample_rate,
      n_mfcc = num_mfccs,
      n_fft = fft_length,
      hop_length = frame_step,
      win_length = frame_length,
      center = False,
      n_mels = num_mel_freqs,
      fmin = fmin_hz,
      fmax = fmax_hz)

### Extract the MFCCs using the Librosa library


In [ ]:
mfccs_librosa = extract_mfccs_librosa(
    test_ad,
    SAMPLE_RATE,
    NUM_MFCCS,
    FRAME_LENGTH,
    FRAME_STEP,
    FFT_LENGTH,
    FMIN_HZ,
    FMAX_HZ,
    NUM_MEL_FREQS)

### Display the MFCCs

In [ ]:
display_mfccs(mfccs_librosa)

## Part 2: Model deployment

## Constants

In [ ]:
# Music genres
LIST_GENRES = ['disco', 'jazz', 'metal']

# Training audio length in seconds
TRAIN_AUDIO_LENGTH_SEC = 1

# Training audio length in number of samples
TRAIN_AUDIO_LENGTH_SAMPLES = SAMPLE_RATE * TRAIN_AUDIO_LENGTH_SEC

# TensorFlow model name
TF_MODEL = 'music_genre'

# TensorFlow lite model name
TFL_MODEL_FILE = TF_MODEL + '_int8' + '.tflite'

## Computing the FFT magnitude with fixed-point arithmetic through the CMSIS-DSP Python library

### Implement a function that computes the RFFT in Q15 fixed-point using CMSIS-DSP.

In [ ]:
def rfft_q15(src):
  src_len = src.shape[0]
  inst = dsp.arm_rfft_instance_q15()
  stat = dsp.arm_rfft_init_q15(inst, src_len, 0, 1)
  fft_q = dsp.arm_rfft_q15(inst, src)
  return fft_q[:src_len + 1]

### Implement a function that computes the magnitude in Q15 fixed-point using CMSIS-DSP

In [ ]:
def mag_q15(src):
  f0 = src[0],
  fn = src[1],
  fx = dsp.arm_cmplx_mag_q15(src[2:])
  return np.concatenate((f0, fx, fn))

### Get a frame from the test audio sample

In [ ]:
src = test_ad[0:FRAME_LENGTH]

### Compute the FFT magnitude using the 16-bit fixed-point arithmetic

In [ ]:
# Convert to Q15
src_q15 = dsp.arm_float_to_q15(src)

# Apply the RFFT. The output is Q12.4. Therefore, fewer fractional bits
cmsis_fft_q15 = rfft_q15(src_q15)

# Calculate the magnitude of the FFT. The output is Q13.3
cmsis_fft_mag_q15 = mag_q15(cmsis_fft_q15)

# Convert to float
scale = float(1 << 3) # 8

cmsis_fft_mag = cmsis_fft_mag_q15 / scale

### Compute the FFT magnitude using the floating-point arithmetic

In [ ]:
tf_fft = tf.signal.rfft(src)
tf_fft_mag = tf.math.abs(tf_fft)

### Evaluate the difference stats

In [ ]:
abs_diff = np.abs(tf_fft_mag - cmsis_fft_mag)
print("Differences:\nmin:", np.min(abs_diff),
      "max:", np.max(abs_diff),
      "mean:", np.mean(abs_diff),
      "std:", np.std(abs_diff))

## Implementing the MFCCs feature extraction with the CMSIS-DSP library


### Implement a function to precompute the Hann window coefficients in Q15 format

In [ ]:
def gen_hann_lut_q15(frame_len):
  hann_lut_f32 = tf.signal.hann_window(frame_len)
  return dsp.arm_float_to_q15(hann_lut_f32)

### Implement a function to precompute the Mel-weight matrix in Q15 format

In [ ]:
def gen_mel_weight_mtx(sr, fmin_hz, fmax_hz, num_mel_freqs, num_fft_freqs):
  m_f32 = tf.signal.linear_to_mel_weight_matrix(
    num_mel_freqs,
    num_fft_freqs,
    sr,
    fmin_hz,
    fmax_hz)

  m_q15 = dsp.arm_float_to_q15(m_f32)
  # Reshape is needed because the conversion from float to q15 collapses the dimensions
  return m_q15.reshape((m_f32.shape[0], m_f32.shape[1]))

### Implement a function to precompute the logarithmic function with input as 16-bit fixed-point

In [ ]:
def gen_log_lut_q(q_scale):
  max_int16 = np.iinfo("int16").max

  log_lut = np.zeros(shape=(max_int16), dtype="int16")

  for i16 in range(0, max_int16):
    q16 = np.array([i16,], dtype="int16")
    f_v = q16 / float(q_scale)
    log_f = np.array(np.log(f_v + 1e-6),)
    log_q = log_f * float(q_scale)
    log_lut[i16] = int(log_q)

  return log_lut

### Implement a function to precompute the DCT-weight matrix in Q15 format

In [ ]:
def gen_dct_weight_mtx(num_mel_freqs, num_mfccs):
  mtx_q15 = np.zeros(shape=(num_mel_freqs, num_mfccs), dtype="int16")

  scale = np.sqrt(2.0 / float(num_mel_freqs))
  pi_div_mel = (math.pi / num_mel_freqs)
  for n in range(num_mel_freqs):
    for k in range(num_mfccs):
      v = scale * np.cos(pi_div_mel * (n + 0.5) * k)
      v_f32 = np.array([v,], dtype="float32")
      mtx_q15[n][k] = dsp.arm_float_to_q15(v_f32)
  return mtx_q15

### Precompute the Hann window coefficients, Mel-weight matrix, DCT-weight matrix, and logarithmic function

In [ ]:
# Generate the Hann Window LUT for Q15
hann_lut_q15 = gen_hann_lut_q15(FRAME_LENGTH)

# Precalculate the Mel-weight matrix in Q15 fixed-point format
mel_wei_mtx_q15 = gen_mel_weight_mtx(SAMPLE_RATE, FMIN_HZ, FMAX_HZ, NUM_MEL_FREQS, int((FFT_LENGTH / 2) + 1))

# Generate the Log LUT for Q13.3 fixed-point format
log_lut_q13_3 = gen_log_lut_q(8)

# Precalculate the DCT-II-weight matrix
dct_wei_mtx_q15 = gen_dct_weight_mtx(NUM_MEL_FREQS, NUM_MFCCS)

### Show the program memory usage

In [ ]:
mem_usage = 0
mem_usage += np.size(hann_lut_q15) * 2
mem_usage += np.size(mel_wei_mtx_q15) * 2
mem_usage += np.size(log_lut_q13_3) * 2
mem_usage += np.size(dct_wei_mtx_q15) * 2

print("Program memory usage: ", mem_usage, "bytes")

### Implement a function to compute the MFCCs feature extraction with 16-bit fixed-point arithmetic

In [ ]:
def extract_mfccs_cmsis(
  ad_src,
  num_mfccs,
  frame_length,
  frame_step,
  hann_lut_q15,
  mel_wei_mtx_q15,
  log_lut_q13_3,
  dct_wei_mtx_q15):

  n = ad_src.shape[0]
  num_frames = int(((n - frame_length) / frame_step) + 1)

  output = np.zeros(shape=(num_frames, num_mfccs))

  # Iterate over each frame to get the MFCC coefficients
  for i in range(num_frames):
    idx_s = i * frame_step
    idx_e = idx_s + frame_length
    frame = ad_src[idx_s:idx_e]

    # Convert to Q15
    frame_q15 = dsp.arm_float_to_q15(frame)

    # Apply the Hann Window. The output is still Q15
    hann_q15 = dsp.arm_mult_q15(frame_q15, hann_lut_q15)

    # Apply the RFFT. The output is Q12.4. Therefore, fewer fractional bits
    fft_spect_q15 = rfft_q15(hann_q15)

    # Calculate the magnitude of the FFT. The output is Q13.3
    fft_mag_spect_q15 = mag_q15(fft_spect_q15)

    # Calculate the Mel-spectrogram
    log_mel_spect_q15 = dsp.arm_mat_vec_mult_q15(mel_wei_mtx_q15.T, fft_mag_spect_q15.T)

    # Calculate the Log-Mel-spectrogram
    for idx, v in enumerate(log_mel_spect_q15):
      log_mel_spect_q15[idx] = log_lut_q13_3[v]

    # Calculate the MFCCs through the DCT
    mfccs = dsp.arm_mat_vec_mult_q15(dct_wei_mtx_q15.T, log_mel_spect_q15)

    # Convert MFCCs to float
    output[i] = mfccs.T / float(8)

  return output

### Extract the MFCC coefficients using the CMSIS-DSP implementation

In [ ]:
mfccs_cmsis = extract_mfccs_cmsis(
    test_ad,
    NUM_MFCCS,
    FRAME_LENGTH,
    FRAME_STEP,
    hann_lut_q15,
    mel_wei_mtx_q15,
    log_lut_q13_3,
    dct_wei_mtx_q15)

### Display MFCCs obtained with the CMSIS-DSP library

In [ ]:
display_mfccs(mfccs_cmsis.T)

### Display the difference

In [ ]:
abs_diff = np.abs(mfccs_tf - mfccs_cmsis)

display_mfccs(abs_diff.T)

print("Differences:\nmin:", np.min(abs_diff),
      "max:", np.max(abs_diff),
      "mean:", np.mean(abs_diff),
      "std:", np.std(abs_diff))